In [13]:
from keras.utils import to_categorical, Sequence
from glob import glob
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np

In [14]:
def import_images(path):
  return (img.imread(path) / 255, to_categorical(int(path.split('/')[-2]), 14))

## Creating the Sequence Object

In [19]:
class SequenceGenerator(Sequence):

  def __init__(self, entries, outputs, batch_size):
    self.entries = entries
    self.entries = outputs
    self.batch_size = batch_size

  def __len__(self):
    return len(self.entries) // self.batch_size

  def __getitem__(self, id):
    esquerda = id * self.batch_size
    direita = min(esquerda + self.batch_size, len(self.entries))
    batch_entries = self.entries[esquerda:direita]
    batch_outputs = self.outputs[esquerda:direita]

    x = np.array([img.imread(caminho)/255 for caminho in batch_entries])
    y = np.array([to_categorical(int(caminho.split('/')[-2]), 14) for caminho in batch_outputs])

    return x, y

In [20]:
paths = glob('./m10/semana_3/imagens/**/*.jpg')

In [21]:
paths

[]